In [8]:
import pandas as pd

# Reading the Data Set: This section loads the dataset and performs initial data exploration:

In [9]:
df = pd.read_csv('client_dataset.csv')

# This command will display all the column names in your dataset:

In [11]:
df.columns

Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')

# This function provides a summary of statistics for each numerical column, including the count, mean, standard deviation, min, max, and quartile values:


In [12]:
df.describe()


,client_id,order_id,order_week,order_year,unit_price,unit_cost,unit_weight,qty,line_number
count,54639.000000,5.463900e+04,54639.000000,54639.000000,54639.000000,54639.000000,54639.000000,5.463900e+04,54639.000000
mean,54837.869416,5.470190e+06,11.359139,2022.993064,136.267207,99.446073,5.004116,5.702646e+02,2.979667
std,25487.438231,2.599807e+06,7.023499,0.082997,183.873135,133.164267,5.326599,1.879552e+04,2.436320
min,10033.000000,1.000886e+06,1.000000,2022.000000,0.010000,0.010000,0.000000,0.000000e+00,0.000000
25%,33593.000000,3.196372e+06,6.000000,2023.000000,20.800000,14.840000,1.450000,3.200000e+01,1.000000
50%,53305.000000,5.496966e+06,11.000000,2023.000000,68.310000,49.890000,3.240000,6.800000e+01,3.000000
75%,78498.000000,7.733869e+06,17.000000,2023.000000,173.160000,125.570000,6.890000,1.700000e+02,5.000000
max,99984.000000,9.998480e+06,52.000000,2023.000000,1396.230000,846.270000,46.430000,3.958244e+06,9.000000


# This will give you a list of the top three categories in your data, sorted by the number of entries:

In [13]:
top_categories = df['category'].value_counts().head(3)
print(top_categories)


category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64


# This identifies the subcategory that appears most frequently within the most common category:

In [14]:
top_category = df['category'].value_counts().idxmax()
top_subcategory = df[df['category'] == top_category]['subcategory'].value_counts().idxmax()
print(top_subcategory)


bathroom supplies


# This shows the client IDs of the top five clients based on the number of entries in the dataset:

In [15]:
top_clients = df['client_id'].value_counts().head(5)
print(top_clients)


client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64


# This creates a list of the top 5 client IDs, which can be used in later analysis:

In [17]:
top_client_ids = top_clients.index.tolist()
print(top_client_ids)


[33615, 66037, 46820, 38378, 24741]


# This calculates the total number of units ordered by the client with the most entries:

In [31]:
top_client_id = top_client_ids[0]  # Get the ID of the client with the most entries
total_units_top_client = df[df['client_id'] == top_client_id]['qty'].sum()
print("\nTotal Units Ordered by the Client with the Most Entries ({}):\n".format(top_client_id), total_units_top_client)



Total Units Ordered by the Client with the Most Entries (33615):
 64313


# The subtotal column represents the total price for each line item before any additional costs like shipping or taxes:

In [19]:
df['subtotal'] = df['unit_price'] * df['qty']


# The shipping_price is calculated per pound based on whether the weight exceeds 50 pounds, and is stored in a new column:

In [20]:
df['shipping_price'] = df['unit_weight'].apply(lambda x: 7 if x > 50 else 10) * df['unit_weight']


# The total_price column represents the final amount billed for each line item, including shipping and tax:

In [21]:
df['total_price'] = (df['subtotal'] + df['shipping_price']) * 1.0925


# The line_cost represents the cost incurred by the company for each line item, including shipping.

In [22]:
df['line_cost'] = (df['unit_cost'] * df['qty']) + df['shipping_price']


# The profit column shows the profit made on each line item after deducting costs:

In [23]:
df['profit'] = df['total_price'] - df['line_cost']


# This code calculates the total price for each of the specified orders, which can be compared to the expected values:

In [24]:
order_ids = [2742071, 2173913, 6128929]
order_totals = df[df['order_id'].isin(order_ids)].groupby('order_id')['total_price'].sum()
print(order_totals)


order_id
2173913    119838.357050
2742071    139844.512025
6128929    752748.745475
Name: total_price, dtype: float64


# This provides the total revenue generated by each of the top 5 clients:

In [25]:
client_revenue = df[df['client_id'].isin(top_client_ids)].groupby('client_id')['total_price'].sum()
print(client_revenue)


client_id
24741    7.667848e+07
33615    6.392918e+06
38378    9.172866e+06
46820    8.006333e+06
66037    8.746255e+06
Name: total_price, dtype: float64


# The summary DataFrame provides a concise overview of the most important metrics for the top 5 clients:

In [26]:
summary_df = df[df['client_id'].isin(top_client_ids)].groupby('client_id').agg({
    'qty': 'sum',
    'shipping_price': 'sum',
    'total_price': 'sum',
    'profit': 'sum'
}).reset_index()
print(summary_df)


   client_id     qty  shipping_price   total_price        profit
0      24741  239862          9365.6  7.667848e+07  3.610666e+07
1      33615   64313         12609.4  6.392918e+06  2.033980e+06
2      38378   73667         11895.0  9.172866e+06  2.955706e+06
3      46820   75768         11094.8  8.006333e+06  2.589495e+06
4      66037   43018         10017.3  8.746255e+06  3.126907e+06


# This step adjusts the data to be in millions for easier presentation and analysis:

In [28]:
def convert_to_millions(value):
    return value / 1_000_000

# Apply the conversion function to each relevant column individually
summary_df['qty'] = summary_df['qty'].apply(convert_to_millions)
summary_df['shipping_price'] = summary_df['shipping_price'].apply(convert_to_millions)
summary_df['total_price'] = summary_df['total_price'].apply(convert_to_millions)
summary_df['profit'] = summary_df['profit'].apply(convert_to_millions)

# Print the result to check
print(summary_df)


   client_id           qty  shipping_price  total_price    profit
0      24741  2.398620e-07    9.365600e-09     0.000077  0.000036
1      33615  6.431300e-08    1.260940e-08     0.000006  0.000002
2      38378  7.366700e-08    1.189500e-08     0.000009  0.000003
3      46820  7.576800e-08    1.109480e-08     0.000008  0.000003
4      66037  4.301800e-08    1.001730e-08     0.000009  0.000003


# This section renames the columns for clarity and sorts the DataFrame by profit in descending order: 

In [29]:
summary_df.columns = ['Client ID', 'Total Units (Millions)', 'Total Shipping Price (Millions)', 'Total Revenue (Millions)', 'Total Profit (Millions)']
summary_df = summary_df.sort_values(by='Total Profit (Millions)', ascending=False)
print(summary_df)


   Client ID  Total Units (Millions)  Total Shipping Price (Millions)  \
0      24741            2.398620e-07                     9.365600e-09   
4      66037            4.301800e-08                     1.001730e-08   
2      38378            7.366700e-08                     1.189500e-08   
3      46820            7.576800e-08                     1.109480e-08   
1      33615            6.431300e-08                     1.260940e-08   

   Total Revenue (Millions)  Total Profit (Millions)  
0                  0.000077                 0.000036  
4                  0.000009                 0.000003  
2                  0.000009                 0.000003  
3                  0.000008                 0.000003  
1                  0.000006                 0.000002  
